In [1]:
!nvidia-smi

Fri Jul 22 14:58:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 23%   29C    P8    16W / 250W |  10913MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 16%   

In [2]:
from jarvis.utils.general import gpus
gpus.find_all()

{0: {'alloc': 10913, 'total': 11178, 'percentage': 0.9762927178386116},
 1: {'alloc': 0, 'total': 11178, 'percentage': 0.0},
 2: {'alloc': 0, 'total': 11178, 'percentage': 0.0},
 3: {'alloc': 0, 'total': 11178, 'percentage': 0.0}}

In [3]:
gpus.find_available()

{1: {'alloc': 0, 'total': 11178, 'percentage': 0.0},
 2: {'alloc': 0, 'total': 11178, 'percentage': 0.0},
 3: {'alloc': 0, 'total': 11178, 'percentage': 0.0}}

In [4]:
gpus.autoselect()

[ 2022-07-22 14:58:31 ] CUDA_VISIBLE_DEVICES automatically set to: 1           


In [6]:
import numpy as np
from jarvis.train import datasets
from jarvis.utils.display import imshow
from tensorflow.keras import Input
from jarvis.train.client import Client

from IPython.display import clear_output
import matplotlib.pyplot as plt
import tensorflow as tf

In [7]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
def show_mri(mri):
    for i in range(mri.shape[0]):
        clear_output(wait=True)
        plt.axis(False)
        plt.imshow(mri[i, :, :, 0] if len(mri.shape) == 4 else mri[i, :, :], cmap='gray')
        plt.show()

## New Version

In [9]:
from ResNet10.ResNet10_3D import QuadInput_ResNet
base_model = QuadInput_ResNet()

In [10]:
bce = tf.keras.losses.BinaryCrossentropy()

In [11]:
base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), loss=bce, metrics=['acc'])

In [12]:
def prep(data):
    xs, ys = data
    inp = np.stack([xs['t2w'], xs['t1w'], xs['t1wce'], xs['fla']], axis=4)
    inp = inp.reshape((4, 48, 96, 96, 4))
    x_prep = (inp-np.min(inp))/(np.max(inp)-np.min(inp))
    if(np.isnan(x_prep).any()):
        x_prep = np.random.normal(size=(4, 48, 96, 96, 4))
    return (x_prep, ys['lbl'].reshape(4, 1))

def prep_quad(data):
    xs, ys = data
    inp = (xs['t2w'].reshape(4, 48, 96, 96), xs['t1w'].reshape(4, 48, 96, 96), xs['t1wce'].reshape(4, 48, 96, 96), xs['fla'].reshape(4, 48, 96, 96))
    x_prep = inp
    if(np.isnan(inp[0]).any() or np.isnan(inp[1]).any() or np.isnan(inp[2]).any() or np.isnan(inp[3]).any()):
        x_prep = (np.random.normal(size=(4, 48, 96, 96)), np.random.normal(size=(4, 48, 96, 96)), np.random.normal(size=(4, 48, 96, 96)), np.random.normal(size=(4, 48, 96, 96)))
    return (x_prep, ys['lbl'].reshape(4, 1))

In [13]:
# --- prepare generators
client = Client('/data/raw/miccai_rsna/data/ymls/client-3d.yml')
gen_train, gen_valid = client.create_generators()
history = base_model.fit(x=map(prep_quad, gen_train), steps_per_epoch=100, epochs=1)

 44/100 [============>.................] - ETA: 16s - loss: 0.7409 - acc: 0.4773

/home/caidm/jarvis/jarvis-core/jarvis/train/client/client.py:983: RuntimeWarning: invalid value encountered in true_divide
  arr_ = arr / scale


100/100 [==============================] - 35s 298ms/step - loss: 0.7239 - acc: 0.4775


In [14]:
base_model.evaluate(x=map(prep_quad, gen_valid), steps=10)

10/10 [==============================] - 3s 283ms/step - loss: 6.8547 - acc: 0.5000


[6.854737281799316, 0.5]

In [24]:
from tqdm import tqdm

In [25]:
NUM_VAL_SCANS = 100

In [27]:
gen_valid_prep = map(prep_quad, gen_valid)
preds = []
labels = []
for i, data in tqdm(enumerate(gen_valid_prep)):
    x, y = data
    pred = base_model.predict(x)
    preds.append(pred)
    labels.append(y.reshape(4, 1))
    if i == NUM_VAL_SCANS:
        break

100it [00:33,  2.99it/s]


In [28]:
preds = np.array(preds).reshape(-1,)
labels = np.array(labels).reshape(-1,)

In [29]:
disc_preds = np.zeros(preds.shape)
disc_preds[preds > 0.5] = 1

In [30]:
from sklearn.metrics import accuracy_score, roc_auc_score

acc = accuracy_score(labels, disc_preds)
auc = roc_auc_score(labels, preds)
print(f'Validated on {NUM_VAL_SCANS} scans...')
print(f'Accuracy: {acc}')
print(f'AUC: {auc}')

Validated on 100 scans...
Accuracy: 0.5074257425742574
AUC: 0.5936059625380014
